In [1]:
import numpy as np
import pandas as pd

In [4]:
DIR = '/local/ruolinzheng/cmsc33750-data/hw4-data/'
NT_TRAIN = 'normal-tumor/nt_train2.csv'
TC_TRAIN = 'type-class/type_18_300_train.csv'

In [3]:
TC_DIR = 'type-class/'

In [7]:
nt_train = pd.read_csv(DIR + NT_TRAIN, nrows=5, header=None)

In [9]:
nt_train

,0,1,2,3,4,5,6,7,8,9,...,60474,60475,60476,60477,60478,60479,60480,60481,60482,60483
0,0,456.733102,0.000000,44807.872348,0.0,79595.148182,166781.014694,0.0,325.147563,2.142157e+06,...,0.0,1444.916307,0.0,2205.989814,761.221837,247138.885268,4660.747965,0.0,10877.082838,243.636741
1,1,0.000000,78.579416,28903.153906,0.0,124497.868573,269812.921183,0.0,2123.340838,2.896222e+06,...,0.0,0.000000,0.0,0.000000,943.875288,626010.838924,666.817367,0.0,10819.253113,0.000000
2,1,0.000000,0.000000,93306.309062,0.0,195027.840224,238776.311467,0.0,3183.091418,3.130360e+06,...,0.0,2525.941418,0.0,5141.890467,0.000000,356687.310095,626.747843,0.0,43462.525691,0.000000
3,1,11211.547894,111.116991,61812.206069,0.0,128086.941127,381735.944206,0.0,1292.240098,1.096682e+07,...,0.0,0.000000,0.0,0.000000,2669.416165,282435.827633,17915.633866,0.0,7125.658632,0.000000
4,1,1861.593617,86.100675,19192.019880,0.0,63179.026516,250062.564291,0.0,161.976896,3.527659e+06,...,0.0,0.000000,0.0,1498.561755,0.000000,219016.974518,2313.699825,0.0,32641.359518,0.000000


In [10]:
tc_train = pd.read_csv(DIR + TC_TRAIN, nrows=5, header=None)

In [11]:
tc_train

,0,1,2,3,4,5,6,7,8,9,...,60474,60475,60476,60477,60478,60479,60480,60481,60482,60483
0,29,13.4,0.0,17.0,0,18.1,17.5,0,0.0,22.3,...,0,0.0,0,0.0,0.0,18.9,12.8,0,13.8,0
1,5,9.4,0.0,15.5,0,15.6,17.7,0,8.0,22.7,...,0,11.0,0,11.1,0.0,18.4,12.3,0,12.8,0
2,29,0.0,0.0,17.5,0,17.3,18.0,0,7.4,21.4,...,0,0.0,0,11.7,0.0,19.0,12.3,0,15.4,0
3,4,12.9,11.3,16.8,0,16.2,17.9,0,4.7,21.1,...,0,6.9,0,9.8,11.8,18.2,13.1,0,14.4,0
4,5,11.0,8.6,16.0,0,16.7,17.8,0,8.2,21.7,...,0,8.3,0,0.0,10.2,18.2,12.5,0,16.5,0


In [4]:
# %load /local/ruolinzheng/cmsc33750-data/hw4-data/type-class/tc1.py
import pandas as pd
import numpy as np
import os
import sys
import gzip

from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

file_path = os.path.dirname(os.path.realpath(__file__))
lib_path = os.path.abspath(os.path.join(file_path, '..', '..', 'common'))
sys.path.append(lib_path)

EPOCH = 400
BATCH = 20
CLASSES = 36

PL = 60484   # 1 + 60483 these are the width of the RNAseq datasets
P     = 60483   # 60483
DR    = 0.1      # Dropout rate

def load_data():
    train_path = 'type_18_300_train.csv'
    test_path = 'type_18_300_test.csv'

    df_train = (pd.read_csv(DIR + TC_DIR + train_path, header=None)\
                .values).astype('float32')
    df_test = (pd.read_csv(DIR + TC_DIR + test_path, header=None)\
               .values).astype('float32')

    print('df_train shape:', df_train.shape)
    print('df_test shape:', df_test.shape)

    df_y_train = df_train[:,0].astype('int')
    df_y_test = df_test[:,0].astype('int')

    Y_train = np_utils.to_categorical(df_y_train,CLASSES)
    Y_test = np_utils.to_categorical(df_y_test,CLASSES)

    df_x_train = df_train[:, 1:PL].astype(np.float32)
    df_x_test = df_test[:, 1:PL].astype(np.float32)

#        X_train = df_x_train.as_matrix()
#        X_test = df_x_test.as_matrix()

    X_train = df_x_train
    X_test = df_x_test

    scaler = MaxAbsScaler()
    mat = np.concatenate((X_train, X_test), axis=0)
    mat = scaler.fit_transform(mat)

    X_train = mat[:X_train.shape[0], :]
    X_test = mat[X_train.shape[0]:, :]

    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = load_data()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

x_train_len = X_train.shape[1]

# this reshaping is critical for the Conv1D to work

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

NameError: name '__file__' is not defined

In [15]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=20, strides=1, padding='valid', input_shape=(P, 1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=10, strides=1, padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=10))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(CLASSES))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

# set up a bunch of callbacks to do work during model training..

checkpointer = ModelCheckpoint(filepath='tc1.autosave.model.h5', verbose=0, save_weights_only=False, save_best_only=True)
csv_logger = CSVLogger('tc1.training.log')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

history = model.fit(X_train, Y_train,
                    batch_size=BATCH,
                    epochs=EPOCH,
                    verbose=1,
                    validation_data=(X_test, Y_test),
                    callbacks = [checkpointer, csv_logger, reduce_lr])

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

# serialize model to JSON
model_json = model.to_json()
with open("tc1.model.json", "w") as json_file:
        json_file.write(model_json)

# serialize model to YAML
model_yaml = model.to_yaml()
with open("tc1.model.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)


# serialize weights to HDF5
model.save_weights("tc1.model.h5")
print("Saved model to disk")

# load json and create model
json_file = open('tc1.model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_json = model_from_json(loaded_model_json)


# load yaml and create model
yaml_file = open('tc1.model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model_yaml = model_from_yaml(loaded_model_yaml)


# load weights into new model
loaded_model_json.load_weights("tc1.model.h5")
print("Loaded json model from disk")

# evaluate json loaded model on test data
loaded_model_json.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score_json = loaded_model_json.evaluate(X_test, Y_test, verbose=0)

print('json Test score:', score_json[0])
print('json Test accuracy:', score_json[1])

print("json %s: %.2f%%" % (loaded_model_json.metrics_names[1], score_json[1]*100))


# load weights into new model
loaded_model_yaml.load_weights("tc1.model.h5")
print("Loaded yaml model from disk")

# evaluate loaded model on test data
loaded_model_yaml.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score_yaml = loaded_model_yaml.evaluate(X_test, Y_test, verbose=0)

print('yaml Test score:', score_yaml[0])
print('yaml Test accuracy:', score_yaml[1])

print("yaml %s: %.2f%%" % (loaded_model_yaml.metrics_names[1], score_yaml[1]*100))

NameError: name 'Sequential' is not defined